In [1]:
import pandas as pd
print(pd.__version__)
import numpy as np

1.4.2


##### Data preprocessing

##### Data preparation

In [ ]:
from data_manip import remove_incomplete_days

df_loadProfiles = pd.read_parquet(r'data/load_profiles.parquet.gzip')   #import data

df_loadProfiles = remove_incomplete_days(df_loadProfiles)

In [ ]:
from sklearn.preprocessing import minmax_scale, MinMaxScaler

profile = '16'

df_profile = df_loadProfiles[['date', 'hour of the day', profile]]
df_profile = df_profile.pivot_table(columns = 'hour of the day', index = 'date', values = profile)

labels = np.array(range(len(df_profile)))
samples = df_profile.to_numpy()

scaler = MinMaxScaler(feature_range = (-1, 1))
samplesScaled = scaler.fit_transform(samples.T).T

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
import torch

dataset = TensorDataset(torch.Tensor(samplesScaled), torch.Tensor(labels))
dataLoader = DataLoader(dataset)

##### Run GAN

In [ ]:
import torch
from torch.utils.data import TensorDataset, DataLoader

if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('GPU is used.')
else:
    device = torch.device('cpu')
    print('CPU is used.')

dimLatent = 32
featureCount = samplesScaled.shape[1]
classCount = len(set(labels))
dimEmbedding = classCount
lr = 2*1e-4/3
maxNorm = 1e6
epochCount = 100
testLabel = 0

In [ ]:
from GAN import GAN

model = GAN(
    device = device,
    dataLoader = dataLoader,
    dimLatent = dimLatent,
    featureCount = featureCount,
    classCount = classCount,
    dimEmbedding = dimEmbedding,
    lr = lr,
    maxNorm = maxNorm,
    epochCount = epochCount,
    testLabel = testLabel
)
model.train()

**Save model**

In [ ]:
torch.save(model, 'models/test_model_2023-11-15.pt')

**Load model**

In [ ]:
model = torch.load('models/test_model_2023-11-15.pt')